# Usage

Tried to make this fairly simple to implement.

In [9]:
!pip install git+https://github.com/jhaberbe/crp

  Cloning https://github.com/jhaberbe/crp to /private/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/pip-req-build-i822sqmd
  Running command git clone --filter=blob:none --quiet https://github.com/jhaberbe/crp /private/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/pip-req-build-i822sqmd
  Resolved https://github.com/jhaberbe/crp to commit eef2c6e5daf2c653941aa398b43d637f2d6f855d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached leidenalg-0.10.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of chinese-restaurant-process to determine which version is compatible with other requirements. This could take a while.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
ERROR: Package 'chinese-restaurant-process'

In [ ]:
from cpr.dirichlet import ChineseRestaurantProcess

In [2]:
import scanpy as sc
adata = sc.read_h5ad("/Users/jameshaberberger/GitHub/chinese-restaurant-process/data/adata.h5ad")
adata = adata[adata.X.sum(axis=1) > 100]

In [7]:
crp = ChineseRestaurantProcess(adata[::100].X.todense())
crp.run(epochs=10)

100%|██████████| 1232/1232 [00:02<00:00, 447.48it/s]


In [8]:
# Predict labels for the full dataset
# You can adjust the `min_membership` parameter to control the minimum membership threshold for assigning labels.
# 1% works well for me. 0% is true CRP, but expect small clusters, which may or may not be representative of reality.
adata.obs["labels"] = crp.predict(adata.X.todense(), min_membership=0.01)

100%|██████████| 123130/123130 [00:56<00:00, 2184.93it/s]
/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/ipykernel_49390/558921049.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["labels"] = crp.predict(adata.X.todense(), min_membership=0.01)
